In [1]:
import torch
from torch.utils.data import Dataset

import os
from PIL import Image
import cv2
import json

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
path  = '/home/aiteam/tykim/scratch/lightweight/my_detection/hold_smartphone/annos/train.json'
with open(path) as f:
    data = json.load(f)

In [30]:
anno = data['annotations'][0]

In [32]:
anno['bbox']

[338, 212, 45, 46]

In [34]:
torch.as_tensor([anno['bbox']], dtype=torch.float32)

tensor([[338., 212.,  45.,  46.]])

In [13]:
anno

{'iscrowd': 0,
 'image_id': 1,
 'bbox': [338, 212, 45, 46],
 'segmentation': [],
 'category_id': 0,
 'id': 1,
 'area': 2070}

In [16]:
torch.tensor(anno['area'], dtype=torch.float32)

tensor(2070.)

In [10]:
img_name = data['images'][0]
img_name

{'height': 284, 'width': 505, 'id': 1, 'file_name': 'a144.jpg'}

In [6]:
boxes = torch.as_tensor(anno['bbox'], dtype=torch.float32)

In [7]:
boxes

tensor([338., 212.,  45.,  46.])

In [17]:
torch.ones((1,), dtype=torch.int64)

tensor([1])

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root, transforms, train):
        self.root = root
        self.tfms = transforms
        # self.imgs = list(sorted(os.listdir(os.path.join(self.root, 'imgs'))))
        if train is True:
            anno_filename = 'train.json'
        else:
            anno_filename = 'val.json'
                
        anno_path = os.path.join(self.root, 'annos', anno_filename)
        with open(anno_path) as f:
            self.annos = json.load(f)
            
    def __len__(self):
        return len(self.annos['annotations'])
    
    def __getitem__(self, idx):
        img_name = self.annos['images'][idx]['file_name']
        img_path = os.path.join(self.root, 'imgs', img_name)
        img = Image.open(img_path).convert("RGB")
        
        anno = self.annos['annotations'][idx]
        boxes = torch.as_tensor([anno['bbox']], dtype=torch.float32)
        
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = torch.ones((1,), dtype=torch.int64)
        target["image_id"] = torch.tensor([self.annos['images'][idx]['id']])
        target["area"] = torch.tensor(anno['area'], dtype=torch.float32)
        target["iscrowd"] = anno['iscrowd']

        if self.tfms is not None:
            img, target = self.tfms(img, target)
            
        return img, target

In [4]:
path = '/home/aiteam/tykim/scratch/lightweight/my_detection/hold_smartphone'
dataset = CustomDataset(path, None, train=True)

In [5]:
i = iter(dataset)

In [8]:
next(i)

(<PIL.Image.Image image mode=RGB size=1200x801>,
 {'boxes': tensor([402., 273., 108., 171.]),
  'labels': tensor([1]),
  'image_id': tensor([3]),
  'area': tensor(18468.),
  'iscrowd': 0})